<a href="https://colab.research.google.com/github/tatudozen/imersao-alura/blob/main/Agente_de_Planejamento_de_Viagens_com_Atra%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%pip -q install google-genai
# Configura a API Key do Google Gemini
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
# Instalar Framework de agentes do Google
!pip install -q google-adk
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import warnings
warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
            final_response += "\n"
    return final_response

def to_markdown(text):
    text = text.replace('•', ' *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [6]:
##########################################
# --- Agente 1: Buscador de Atrações --- #
##########################################
def agente_buscador_atracoes(destino):
    buscador_atracoes = Agent(
        name="agente_buscador_atracoes",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em encontrar as principais atrações turísticas de um destino.
        Com base no destino fornecido, você irá buscar e listar as atrações imperdíveis,
        incluindo informações relevantes como:

        1.  **Pesquise Atrações:** Utilize a ferramenta de busca do Google (google_search)
            para encontrar as principais atrações turísticas do destino. Considere:
            -   Pontos turísticos famosos e históricos.
            -   Museus, galerias de arte e centros culturais.
            -   Parques, praias e áreas naturais.
            -   Eventos, festivais e atividades sazonais.
            -   Atividades noturnas e de entretenimento.

        2.  **Organize as Atrações:** Ordene as atrações por relevância e popularidade.

        3.  **Formato de Saída:** Retorne as informações sobre as atrações no seguinte formato JSON:
        ```json
        {
            "destino": "Nome do Destino",
            "atracoes": [
                {
                    "nome": "Nome da Atração",
                    "descricao": "Descrição da Atração",
                    "localizacao": "Localização da Atração",
                    "tempo_visita_estimado": "Tempo Estimado de Visita",
                    "avaliacao": "Avaliação dos Visitantes (se disponível)",
                    "link_oficial": "Link para o Site Oficial (se disponível)"
                },
                // ... outras atrações
            ]
        }
        ```
        """,
        tools=[google_search]
    )
    entrada_do_agente = f"Destino: {destino}"
    atracoes_json = call_agent(buscador_atracoes, entrada_do_agente)
    return atracoes_json

In [7]:
atracoes= agente_buscador_atracoes("Turquia")
print(atracoes)

```json
{
    "destino": "Turquia",
    "atracoes": [
        {
            "nome": "Istambul",
            "descricao": "A maior cidade da Turquia, situada entre a Europa e a Ásia, com uma rica história e influências bizantinas e otomanas. Principais pontos turísticos incluem a Hagia Sophia, a Mesquita Azul, o Palácio de Topkapi, o Grande Bazar e um passeio pelo Bósforo.",
            "localizacao": "Turquia",
            "tempo_visita_estimado": "Vários dias",
            "avaliacao": "Excelente",
            "link_oficial": null
        },
        {
            "nome": "Capadócia",
            "descricao": "Região famosa pelas formações rochosas únicas, cidades subterrâneas e passeios de balão ao nascer do sol. Inclui o Göreme Open Air Museum, o Vale dos Monges e o Vale do Amor.",
            "localizacao": "Região de Anatólia Central, Turquia",
            "tempo_visita_estimado": "2-3 dias",
            "avaliacao": "Excelente",
            "link_oficial": null
        },
        

In [9]:
##########################################
# --- Agente 2: Planejador de Roteiro --- #
##########################################
def agente_planejador_roteiro(origem, destino, atracoes):
    planejador_roteiro = Agent(
        name="agente_planejador_roteiro",
        model="gemini-2.0-flash",
        instruction="""
         Você é um planejador de viagens especializado em criar roteiros personalizados para um destino específico,
        levando em conta a origem da viagem e as atrações turísticas disponíveis. Com base na origem, destino
        e atrações, você irá gerar um roteiro detalhado que inclua:

        1.  **Analise as Atrações:** Considere a lista de atrações fornecida, incluindo sua localização,
            tempo de visita estimado e relevância.

        2.  **Roteiro Diário:** Crie um roteiro diário detalhado, organizando as atrações de forma lógica
            e eficiente. Considere o tempo de deslocamento entre as atrações e agrupe atrações próximas
            no mesmo dia. Inclua:
            -   Dia 0: Deslocamento da origem para o destino (se aplicável).
            -   Dias 1, 2, ...: Roteiro detalhado com Manhã, Tarde e Noite, incluindo descrição
                das atividades, tempo estimado e atração correspondente.
            -   Dia de Retorno: Deslocamento do destino para a origem (se aplicável).
            -   Sugestões de restaurantes (se possível, com links) próximos às atrações do dia.
            -   Tempo de deslocamento entre as atividades.

        3.  **Necessidade de Deslocamento Adicional:** Determine se há necessidade de deslocamento adicional
            para atividades específicas dentro do roteiro. Por exemplo, se o destino principal é Istambul,
            mas o roteiro inclui um passeio de balão na Capadócia, você deve indicar a necessidade
            de passagens e acomodação para a Capadócia.

        4.  **Formato de Saída:** Retorne o roteiro no seguinte formato JSON:
        ```json
        {
            "origem": "Cidade de Origem",
            "destino": "Nome do Destino",
            "roteiro": [
                {
                    "dia": 0,
                    "descricao": "Deslocamento da origem para o destino",
                    "tempo_estimado": "Tempo Estimado de Deslocamento"
                },
                {
                    "dia": 1,
                    "manhã": {
                        "atividade": "Nome da Atividade",
                        "descricao": "Descrição da Atividade",
                        "tempo_estimado": "Tempo Estimado",
                        "atracao": "Nome da Atração Correspondente"
                    },
                    "tarde": {
                        "atividade": "Nome da Atividade",
                        "descricao": "Descrição da Atividade",
                        "tempo_estimado": "Tempo Estimado",
                        "atracao": "Nome da Atração Correspondente"
                    },
                    "noite": {
                        "atividade": "Nome da Atividade",
                        "descricao": "Descrição da Atividade"
                    },
                    "restaurantes": [
                        {"nome": "Nome do Restaurante", "link": "Link do Restaurante"},
                        // ...
                    ],
                    "deslocamento": "Tempo de Deslocamento"
                },
                // ... para cada dia
                {
                    "dia": "Retorno",
                    "descricao": "Deslocamento do destino para a origem",
                    "tempo_estimado": "Tempo Estimado de Deslocamento"
                }
            ],
            "deslocamento_adicional": [
                {
                    "local": "Nome do Local",
                    "necessidade_passagem": true/false,
                    "necessidade_acomodacao": true/false,
                    "justificativa": "Justificativa para o deslocamento adicional",
                    "passagens": [
                        {
                            "tipo": "Tipo de Transporte",
                            "empresa": "Nome da Empresa",
                            "preco": "Preço da Passagem",
                            "link_compra": "Link para Compra"
                        }
                    ],
                    "acomodacoes": [
                        {
                            "nome": "Nome da Acomodação",
                            "descricao": "Descrição",
                            "preco": "Preço Médio",
                            "link_reserva": "Link para Reserva"
                        }
                    ]
                }
                // ... outros deslocamentos adicionais
            ]
        }
        ```
        """,
        tools=[google_search]
    )
    entrada_do_agente = f"Origem: {origem}\nDestino: {destino}\nAtrações: {atracoes}"
    roteiro_json = call_agent(planejador_roteiro, entrada_do_agente)
    return roteiro_json

In [11]:
roteiro= agente_planejador_roteiro("Rio de Janeiro", "Instambul", atracoes)
print(roteiro)

```json
{
    "origem": "Rio de Janeiro",
    "destino": "Istambul",
    "roteiro": [
        {
            "dia": 0,
            "descricao": "Deslocamento do Rio de Janeiro para Istambul",
            "tempo_estimado": "Aproximadamente 16-19 horas de voo, dependendo da companhia aérea e escalas."
        },
        {
            "dia": 1,
            "manhã": {
                "atividade": "Visita à Hagia Sophia",
                "descricao": "Explore a magnífica Hagia Sophia, um dos monumentos mais icônicos do mundo, com sua arquitetura deslumbrante e rica história.",
                "tempo_estimado": "2-3 horas",
                "atracao": "Hagia Sophia"
            },
            "tarde": {
                "atividade": "Visita à Mesquita Azul",
                "descricao": "Admire a beleza da Mesquita Azul, famosa por seus seis minaretes e interior decorado com azulejos azuis de Iznik.",
                "tempo_estimado": "2-3 horas",
                "atracao": "Mesquita Azul"
    

In [14]:
##########################################
# --- Agente 3: Buscador de Acomodações --- #
##########################################
def agente_buscador_acomodacoes(roteiro, data_ida, data_volta):
    buscador_acomodacoes = Agent(
        name="agente_buscador_acomodacoes",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em encontrar as melhores acomodações para viagens.
        Com base no roteiro de viagem fornecido, você irá buscar e sugerir opções de
        hotéis e outros tipos de acomodação que sejam adequados para cada etapa da viagem.
        Leve em consideração as datas da viagem para estimar os preços.

        1.  **Extraia o Destino e Deslocamentos:** Analise o roteiro para identificar o destino principal
            da viagem e quaisquer deslocamentos adicionais.

        2.  **Calcule o Número de Diárias:** Calcule o número total de diárias da viagem,
            considerando a data de ida e a data de volta.

        3.  **Busque Acomodações:** Utilize a ferramenta de busca do Google (google_search)
            para encontrar hotéis, pousadas, apartamentos e outras opções de acomodação
            relevantes para o destino principal e deslocamentos adicionais.  Inclua informações como:
            -   Nome da acomodação.
            -   Descrição breve.
            -   Link para a página de reservas.
            -   Preço médio por noite (estimado com base nas datas da viagem).

        4.  **Formato de Saída:** Retorne as informações sobre as acomodações no seguinte
            formato JSON:
        ```json
        {
            "destino": "Nome do Destino",
            "num_diarias": "Número Total de Diárias",
            "acomodacoes": [
                {
                    "local": "Cidade/Local",
                    "opcoes": [
                        {"nome": "Nome da Acomodação 1", "descricao": "Descrição da Acomodação 1", "link": "Link 1", "preco_diaria": "Preço da Diária 1"},
                        {"nome": "Nome da Acomodação 2", "descricao": "Descrição da Acomodação 2", "link": "Link 2", "preco_diaria": "Preço da Diária 2"},
                        // ...
                    ]
                },
                // ... para outros locais de deslocamento adicional (se houver)
            ]
        }
        ```
        """,
        tools=[google_search]
    )
    entrada_do_agente = f"Roteiro de viagem: {roteiro}\nData de Ida: {data_ida}\nData de Volta: {data_volta}"
    acomodacoes_json = call_agent(buscador_acomodacoes, entrada_do_agente)
    return acomodacoes_json

In [15]:
acomodacoes= agente_buscador_acomodacoes(roteiro)
print(acomodacoes)

NameError: name 'data_ida' is not defined

In [16]:
##########################################
# --- Agente 4: Buscador de Passagens --- #
##########################################
def agente_buscador_passagens(roteiro, data_ida, data_volta):
    buscador_passagens = Agent(
        name="agente_buscador_passagens",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em encontrar as melhores opções de transporte e passagens
        para viagens. Com base no roteiro de viagem fornecido, e nas datas de início e fim da viagem,
        você irá buscar e sugerir as melhores opções de voos, trens, ônibus ou outros meios de
        transporte disponíveis, focando nos deslocamentos necessários entre a origem,
        destino principal e quaisquer deslocamentos adicionais.

        1.  **Entenda os Parâmetros:** Obtenha o roteiro de viagem e as datas de ida e volta.

        2.  **Analise o Roteiro:**
            -   Identifique a origem e o destino principal da viagem.
            -   Verifique se há deslocamentos adicionais no roteiro (e.g., de Istambul para a Capadócia).
            -   Determine as datas de deslocamento para cada trecho da viagem (ida para o destino,
                deslocamentos adicionais, retorno da origem).

        3.  **Busque Opções de Transporte:** Utilize a ferramenta de busca do Google (google_search)
            para encontrar opções de transporte para cada trecho identificado. Considere:
            -   Vários meios de transporte (voos, trens, ônibus, etc.).
            -   Diferentes empresas e horários.
            -   Preços e tempo de viagem.
            -   Conexões e escalas (se houver).

        4.  **Filtre e Ordene:** Filtre as opções de transporte com base nos seguintes critérios:
            -   Menor preço.
            -   Menor tempo de viagem.
            -   Melhor custo-benefício.
            -   Preferências do usuário (se fornecidas).

        5.  **Formato de Saída:** Retorne as informações sobre as passagens no seguinte formato JSON:
        ```json
        {
            "passagens": [
                {
                    "trecho": "Origem - Destino (Data)",
                    "opcoes_transporte": [
                        {
                            "tipo": "Tipo de Transporte",
                            "empresa": "Nome da Empresa",
                            "horario_ida": "Horário de Ida",
                            "horario_volta": "Horário de Volta",
                            "preco": "Preço da Passagem",
                            "tempo_viagem": "Tempo de Viagem",
                            "escalas": "Número de Escalas (se houver)",
                            "link_compra": "Link para Comprar a Passagem"
                        },
                        // ... outras opções de transporte para este trecho
                    ]
                },
                // ... para outros trechos (deslocamentos adicionais, retorno)
            ]
        }
        ```
        """,
        tools=[google_search]
    )
    entrada_do_agente = f"Roteiro de viagem: {roteiro}\nData de Ida: {data_ida}\nData de Volta: {data_volta}"
    passagens_json = call_agent(buscador_passagens, entrada_do_agente)
    return passagens_json

In [14]:
##########################################
# --- Agente 5: Planejador de Viagens Completo --- #
##########################################
def agente_planejador_viagens(origem, destino, data_ida, data_volta, num_dias, tipo_viagem):
    # 1. Busca as Atrações
    atracoes = agente_buscador_atracoes(destino)

    # 2.  Planeja o Roteiro
    roteiro = agente_planejador_roteiro(origem, destino, atracoes)

    # 3. Verifica necessidade de deslocamento adicional e busca passagens e acomodações se necessário
    deslocamentos_adicionais = roteiro.get("deslocamento_adicional", [])
    for deslocamento in deslocamentos_adicionais:
        if deslocamento["necessidade_passagem"]:
            passagens_adicionais = agente_buscador_passagens(origem, deslocamento["local"], data_ida, data_volta) # TODO: ajustar datas
            deslocamento["passagens"] = passagens_adicionais
        if deslocamento["necessidade_acomodacao"]:
            acomodacoes_adicionais = agente_buscador_acomodacoes(roteiro) # TODO: filtrar roteiro
            deslocamento["acomodacoes"] = acomodacoes_adicionais

    # 4. Busca as Acomodações para o destino principal
    roteiro_com_acomodacoes = agente_buscador_acomodacoes(roteiro)

    # 5. Busca as Passagens para o destino principal
    passagens = agente_buscador_passagens(origem, destino, data_ida, data_volta)

    # 6.  Combina tudo em um roteiro completo
    roteiro_completo = {
        "origem": origem,
        "destino": destino,
        "num_dias": num_dias,
        "tipo_viagem": tipo_viagem,
        "passagens": passagens,
        "roteiro": roteiro_com_acomodacoes["roteiro"],
        "deslocamentos_adicionais": deslocamentos_adicionais
    }
    return roteiro_completo

In [16]:
if __name__ == "__main__":
    print("✈️ Bem-vindo ao Agente de Planejamento de Viagens Personalizado! ✈️")
    origem = input("❓ De onde você vai partir? ")
    destino = input("❓ Para onde você gostaria de viajar? ")
    data_ida = input("❓ Qual a data de ida? (DD/MM/AAAA) ")
    data_volta = input("❓ Qual a data de volta? (DD/MM/AAAA) ")
    num_dias = int(input("❓ Quantos dias você tem disponíveis no destino? "))
    tipo_viagem = input("❓ Que tipo de viagem você prefere? (Romântica, Aventura, Família, etc.) ")

    roteiro_completo = agente_planejador_viagens(origem, destino, data_ida, data_volta, num_dias, tipo_viagem)
    print("\n--- 📝 Roteiro de Viagem Completo ---\n")
    #display(to_markdown(roteiro_completo))
    print(roteiro_completo)
    print("--------------------------------------------------------------")


✈️ Bem-vindo ao Agente de Planejamento de Viagens Personalizado! ✈️
❓ De onde você vai partir? Rio de Janeiro
❓ Para onde você gostaria de viajar? Istambul
❓ Qual a data de ida? (DD/MM/AAAA) 20/5/2025
❓ Qual a data de volta? (DD/MM/AAAA) 03/6/2025
❓ Quantos dias você tem disponíveis no destino? 12
❓ Que tipo de viagem você prefere? (Romântica, Aventura, Família, etc.) Aventura


AttributeError: 'str' object has no attribute 'get'

# Nova seção

""",
        tools=[google_search]
    )
    entrada_do_agente = f"Destino: {destino}"
    atracoes_json = call_agent(buscador_atracoes, entrada_do_agente)
    return atracoes_json

##########################################
# --- Agente 2: Planejador de Roteiro --- #
##########################################
def agente_planejador_roteiro(origem, destino, atracoes):
    planejador_roteiro = Agent(
        name="agente_planejador_roteiro",
        model="gemini-1.5-pro-preview-0515",
        instruction="""
        Você é um planejador de viagens especializado em criar roteiros personalizados para um destino específico,
        levando em conta a origem da viagem e as atrações turísticas disponíveis. Com base na origem, destino
        e atrações, você irá gerar um roteiro detalhado que inclua:

        1.  **Analise as Atrações:** Considere a lista de atrações fornecida, incluindo sua localização,
            tempo de visita estimado e relevância.

        2.  **Roteiro Diário:** Crie um roteiro diário detalhado, organizando as atrações de forma lógica
            e eficiente. Considere o tempo de deslocamento entre as atrações e agrupe atrações próximas
            no mesmo dia. Inclua:
            -   Manhã: Atividade 1, com descrição, tempo estimado e atração correspondente.
            -   Tarde: Atividade 2, com descrição, tempo estimado e atração correspondente.
            -   Noite: Atividade 3 (jantar, show, etc.), com descrição e possível atração relacionada.
            -   Sugestões de restaurantes (se possível, com links) próximos às atrações do dia.
            -   Tempo de deslocamento entre as atividades.

        3.  **Necessidade de Deslocamento Adicional:** Determine se há necessidade de deslocamento adicional
            para atividades específicas dentro do roteiro. Por exemplo, se o destino principal é Istambul,
            mas o roteiro inclui um passeio de balão na Capadócia, você deve indicar a necessidade
            de passagens e acomodação para a Capadócia.

        4.  **Formato de Saída:** Retorne o roteiro no seguinte formato JSON:
        ```json
        {
            "destino": "Nome do Destino",
            "roteiro": [
                {
                    "dia": 1,
                    "manhã": {
                        "atividade": "Nome da Atividade",
                        "descricao": "Descrição da Atividade",
                        "tempo_estimado": "Tempo Estimado",
                        "atracao": "Nome da Atração Correspondente"
                    },
                    "tarde": {
                        "atividade": "Nome da Atividade",
                        "descricao": "Descrição da Atividade",
                        "tempo_estimado": "Tempo Estimado",
                        "atracao": "Nome da Atração Correspondente"
                    },
                    "noite": {
                        "atividade": "Nome da Atividade",
                        "descricao": "Descrição da Atividade"
                    },
                    "restaurantes": [
                        {"nome": "Nome do Restaurante", "link": "Link do Restaurante"},
                        // ...
                    ],
                    "deslocamento": "Tempo de Deslocamento"
                },
                // ... para cada dia
            ],
            "deslocamento_adicional": [
                {
                    "local": "Nome do Local",
                    "necessidade_passagem": true/false,
                    "necessidade_acomodacao": true/false,
                    "justificativa": "Justificativa para o deslocamento adicional"
                }
                // ... outros deslocamentos adicionais
            ]
        }
        ```
        """,
        tools=[google_search]
    )
    entrada_do_agente = f"Origem: {origem}\nDestino: {destino}\nAtrações: {atracoes}"
    roteiro_json = call_agent(planejador_roteiro, entrada_do_agente)
    return roteiro_json

##########################################
# --- Agente 3: Buscador de Acomodações --- #
##########################################
def agente_buscador_acomodacoes(roteiro):
    buscador_acomodacoes = Agent(
        name="agente_buscador_acomodacoes",
        model="gemini-1.5-pro-preview-0515",
        instruction="""
        Você é um especialista em encontrar as melhores acomodações para viagens.
        Com base no roteiro de viagem fornecido, você irá buscar e sugerir opções de
        hotéis e outros tipos de acomodação que sejam adequados para cada etapa da viagem.

        1.  **Extraia o Destino:** Analise o roteiro para identificar o destino da viagem.

        2.  **Considere o Roteiro Diário:** Para cada dia do roteiro, leve em conta:
            -   A localização das atividades planejadas (para sugerir acomodações próximas).
            -   O tipo de viagem (para sugerir acomodações com o estilo apropriado:
                romântico, familiar, etc.).
            -   O orçamento (se disponível no roteiro, senão, sugira opções em diferentes faixas
                de preço).

        3.  **Busque Acomodações:** Utilize a ferramenta de busca do Google (google_search)
            para encontrar hotéis, pousadas, apartamentos e outras opções de acomodação
            relevantes. Inclua informações como:
            -   Nome da acomodação.
            -   Descrição breve.
            -   Link para a página de reservas.
            -   Preço médio por noite.
            -   Avaliação dos hóspedes (se disponível).

        4.  **Formato de Saída:** Retorne as informações sobre as acomodações no seguinte
            formato JSON, mantendo a estrutura do roteiro original e adicionando as acomodações:
        ```json
        {
            "destino": "Nome do Destino",
            "num_dias": Número de Dias,
            "tipo_viagem": "Tipo de Viagem",
            "roteiro": [
                {
                    "dia": 1,
                    "manhã": { ... },
                    "tarde": { ... },
                    "noite": { ... },
                    "restaurantes": [ ... ],
                    "deslocamento": "...",
                    "acomodacoes": [  // Novo array de acomodações para o dia
                        {"nome": "Nome da Acomodação 1", "descricao": "Descrição da Acomodação 1", "link": "Link 1", "preco": "Preço 1", "avaliacao": "Avaliação 1"},
                        {"nome": "Nome da Acomodação 2", "descricao": "Descrição da Acomodação 2", "link": "Link 2", "preco": "Preço 2", "avaliacao": "Avaliação 2"},
                        // ...
                    ]
                },
                // ... para cada dia
            ]
        }
        ```
        """,
        tools=[google_search]
    )
    entrada_do_agente = f"Roteiro de viagem: {roteiro}"
    acomodacoes_json = call_agent(buscador_acomodacoes, entrada_do_agente)
    return acomodacoes_json

##########################################
# --- Agente 4: Buscador de Passagens --- #
##########################################
def agente_buscador_passagens(origem, destino, data_ida, data_volta):
    buscador_passagens = Agent(
        name="agente_buscador_passagens",
        model="gemini-1.5-pro-preview-0515",
        instruction="""
        Você é um especialista em encontrar as melhores opções de transporte e passagens
        para viagens. Com base na origem, destino, data de ida e data de volta, você irá
        buscar e sugerir as melhores opções de voos, trens, ônibus ou outros meios de
        transporte disponíveis.

        1.  **Entenda os Parâmetros:** Obtenha a origem, destino, data de ida e data de volta da viagem.

        2.  **Busque Opções de Transporte:** Utilize a ferramenta de busca do Google (google_search)
            para encontrar opções de transporte. Considere:
            -   Vários meios de transporte (voos, trens, ônibus, etc.).
            -   Diferentes empresas e horários.
            -   Preços e tempo de viagem.
            -   Conexões e escalas (se houver).

        3.  **Filtre e Ordene:** Filtre as opções de transporte com base nos seguintes critérios:
            -   Menor preço.
            -   Menor tempo de viagem.
            -   Melhor custo-benefício.
            -   Preferências do usuário (se fornecidas).

        4.  **Formato de Saída:** Retorne as informações sobre as passagens no seguinte formato JSON:
        ```json
        {
            "origem": "Cidade de Origem",
            "destino": "Cidade de Destino",
            "data_ida": "Data de Ida",
            "data_volta": "Data de Volta",
            "opcoes_transporte": [
                {
                    "tipo": "Tipo de Transporte (Voo, Trem, Ônibus)",
                    "empresa": "Nome da Empresa",
                    "horario_ida": "Horário de Ida",
                    "horario_volta": "Horário de Volta",
                    "preco": "Preço da Passagem",
                    "tempo_viagem": "Tempo de Viagem",
                    "escalas": "Número de Escalas (se houver)",
                    "link_compra": "Link para Comprar a Passagem"
                },
                // ... outras opções de transporte
            ]
        }
        ```
        """,
        tools=[google_search]
    )
    entrada_do_agente = f"Origem: {origem}\nDestino: {destino}\nData de Ida: {data_ida}\nData de Volta: {data_volta}"
    passagens_json = call_agent(buscador_passagens, entrada_do_agente)
    return passagens_json

##########################################
# --- Agente 5: Planejador de Viagens Completo --- #
##########################################
def agente_planejador_viagens(origem, destino, data_ida, data_volta, num_dias, tipo_viagem):
    # 1. Busca as Atrações
    atracoes = agente_buscador_atracoes(destino)

    # 2.  Planeja o Roteiro
    roteiro = agente_planejador_roteiro(origem, destino, atracoes)

    # 3. Verifica necessidade de deslocamento adicional e busca passagens e acomodações se necessário
    deslocamentos_adicionais = roteiro.get("deslocamento_adicional", [])
    for deslocamento in deslocamentos_adicionais:
        if deslocamento["necessidade_passagem"]:
            passagens_adicionais = agente_buscador_passagens(origem, deslocamento["local"], data_ida, data_volta) # TODO: ajustar datas
            deslocamento["passagens"] = passagens_adicionais
        if deslocamento["necessidade_acomodacao"]:
            acomodacoes_adicionais = agente_buscador_acomodacoes(roteiro) # TODO: filtrar roteiro
            deslocamento["acomodacoes"] = acomodacoes_adicionais

    # 4. Busca as Acomodações para o destino principal
    roteiro_com_acomodacoes = agente_buscador_acomodacoes(roteiro)

    # 5. Busca as Passagens para o destino principal
    passagens = agente_buscador_passagens(origem, destino, data_ida, data_volta)

    # 6.  Combina tudo em um roteiro completo
    roteiro_completo = {
        "origem": origem,
        "destino": destino,
        "num_dias": num_dias,
        "tipo_viagem": tipo_viagem,
        "passagens": passagens,
        "roteiro": roteiro_com_acomodacoes["roteiro"],
        "deslocamentos_adicionais": deslocamentos_adicionais
    }
    return roteiro_completo

if __name__ == "__main__":
    print("✈️ Bem-vindo ao Agente de Planejamento de Viagens Personalizado! ✈️")
    origem = input("❓ De onde você vai partir? ")
    destino = input("❓ Para onde você gostaria de viajar? ")
    data_ida = input("❓ Qual a data de ida? (DD/MM/AAAA) ")
    data_volta = input("❓ Qual a data de volta? (DD/MM/AAAA) ")
    num_dias = int(input("❓ Quantos dias você tem disponíveis no destino? "))
    tipo_viagem = input("❓ Que tipo de viagem você prefere? (Romântica, Aventura, Família, etc.) ")

    roteiro_completo = agente_planejador_viagens(origem, destino, data_ida, data_volta, num_dias, tipo_viagem)
    print("\n--- 📝 Roteiro de Viagem Completo ---\n")
    display(to_markdown(roteiro_completo))
    print("--------------------------------------------------------------")

  ```